In [114]:
import pandas as pd
import numpy as np 
import choix
import warnings
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


In [115]:
data = pd.read_csv("/Users/louzhou/Downloads/datathon_2024_dataset (1).csv")

In [116]:
data['game_date'] = data['game_date'].astype(str)
data['year'] = data['game_date'].str[:4].astype(int)
data['month'] = data['game_date'].str[4:6].astype(int)
data['day'] = data['game_date'].str[6:8].astype(int)

In [117]:
def getwinner(data):
    if data["score_diff"] > 0:
        output = [data["home_team_index"], data["away_team_index"
                                                ]]
    else:
        output = [data["away_team_index"], data["home_team_index"]]
    return output
def getindexlist(data):
    output = [data["home_team_index"], data["away_team_index"]]
    return output

In [118]:
def getoddswinner(data):
    probhomewin, probawaywin = choix.probabilities(data["home,away"], strength)
    return probhomewin


In [119]:
def get_bt_winner(data):
    teamindex = {}
    index = 0
    for team in data["home_team"].unique():
        teamindex[team] = index
        index = index + 1
    data["home_team_index"] = data["home_team"].map(teamindex)
    data["away_team_index"] = data["away_team"].map(teamindex)
    data["score_diff"] = data["home_score"] - data["away_score"]
    data["cleaned"] = data.apply(getwinner, axis = 1)
    data["home,away"] = data.apply(getindexlist, axis = 1)
    bt_data = data["cleaned"].tolist()
    numteams = len(data["home_team"].unique())
    strength = choix.ilsr_pairwise(numteams, bt_data, alpha=0.01)
    data["homewinp"] = data.apply(getoddswinner, axis = 1)
    return data
    

In [120]:
final_data = pd.DataFrame()
for year in data["year"].unique():
    seasondf = data[data["year"] == year]
    season_bt = get_bt_winner(seasondf)
    final_data = pd.concat([final_data, season_bt])


In [121]:
final_data["homewinp"]

0        0.668397
1        0.331603
2        0.571591
3        0.564251
4        0.391873
           ...   
56770    0.452701
56771    0.419913
56772    0.365621
56773    0.618385
56774    0.559555
Name: homewinp, Length: 56775, dtype: float64

In [122]:
data["score_diff"] = data["home_score"] - data["away_score"]
data["score_diff"].std()

4.399392654950479

In [123]:
data["score_diff"].mean()

0.13076177895200353

In [124]:
def winner(data):
    if data["score_diff"] > 0:
        return 1
    else:
        return 0
final_data["winner"] = final_data.apply(winner, axis = 1)
final_data["winner"]


0        0
1        0
2        1
3        0
4        1
        ..
56770    0
56771    1
56772    1
56773    1
56774    0
Name: winner, Length: 56775, dtype: int64

In [125]:
final_data[["winner", "homewinp"]]
from sklearn.metrics import log_loss, mean_squared_error, roc_auc_score
roc_auc_score(final_data["winner"], final_data["homewinp"])



0.5189047466368881